In [1]:
!pip install fuzzywuzzy
!pip install keras_visualizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import re
from fuzzywuzzy import fuzz
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import activations
from difflib import get_close_matches

/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# dictionaries 
en_letters = [' ','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
ru_letters = [' ','а','б','в','г','д','е','ё','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ъ','ы','ь','э','ю','я']


In [4]:
# load data-set
data =  pd.read_csv('/content/crosswordsfiltered.csv')

data['ru'] = data['ru'].astype('str')
data['en'] = data['en'].astype('str')

data.describe()

,ru,en
count,931,931
unique,855,851
top,пластик,moment
freq,3,3


In [5]:
lang_en =  pd.read_csv('/content/top1000en.csv')
mask = (lang_en["the"].str.len() > 2) 
patterns = lang_en.loc[mask]
patterns = patterns["the"].to_numpy()
len(patterns)



9576

In [6]:
# encode words to (3,) array for input chars and (1,) array  for output chars
def encode_dataset(lang_data,dic,target_lang_data,target_dic):
   data = []
   target = []
   for index,word in enumerate(lang_data):
      word = word.strip()
      target_word = target_lang_data[index]
      if len(target_word) > len(word) : 
         target_word = target_word[:len(word)]
      elif len(target_word) < len(word) : 
         word = word[:len(target_word)]

      target_word = target_word.strip()
      word = ' '+word.lower()+' '
      target_word = target_word.lower()

      for indx,ltr in enumerate(word):
            if ltr!=' ':
                row = [0] * 3
                for i,ltr in enumerate(word[(indx-1):(indx+2)]):
                    if ltr in dic:
                        row[i] =   dic.index(ltr)
                data.append(row) 
                target.append(target_dic.index(target_word[indx-1]))                    
   return data,target   

In [7]:
def encode_dataset2(lang_data,dic,target_lang_data,target_dic):
   data = []
   target = []
   for index,word in enumerate(lang_data):
      word = word.strip()
      target_word = target_lang_data[index]
      if len(target_word) > len(word) : 
         word = word+(' '* (len(target_word)-len(word)))
      elif len(target_word) < len(word) : 
          target_word = target_word+(' '* (len(word)-len(target_word)))

     # target_word = target_word.strip()
      word = ' '+word.lower()+' '
      target_word = target_word.lower()

      for indx,ltr in enumerate(word):
            if ltr!=' ':
                row = [0] * 3
                for i,ltr in enumerate(word[(indx-1):(indx+2)]):
                    if ltr in dic:
                        row[i] =   dic.index(ltr)
                data.append(row) 
                target.append(target_dic.index(target_word[indx-1]))                    
   return data,target 

In [8]:

model = keras.models.Sequential([
    # layers.InputLayer(3,),  
    layers.Embedding(input_dim=34,output_dim=3,input_length=3), 
    layers.Flatten(),     
    layers.Dense(81,activation=activations.relu), 
    layers.Dense(27, activation="softmax")  
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 3)              102       
                                                                 
 flatten (Flatten)           (None, 9)                 0         
                                                                 
 dense (Dense)               (None, 81)                810       
                                                                 
 dense_1 (Dense)             (None, 27)                2214      
                                                                 
Total params: 3,126
Trainable params: 3,126
Non-trainable params: 0
_________________________________________________________________


In [9]:
# model config
model_loss = keras.losses.MeanAbsoluteError()
# model.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy())
model.compile(loss="categorical_crossentropy", optimizer= "adam", metrics=['accuracy'])

In [10]:
#data prepration / encoded 
X_train,Y_train = encode_dataset2(data["ru"],ru_letters,data["en"],en_letters)
X_train = np.asarray(X_train)
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=27)
X_train

array([[ 0, 19,  6],
       [19,  6,  4],
       [ 6,  4, 14],
       ...,
       [ 1,  9, 10],
       [ 9, 10, 33],
       [10, 33,  0]])

In [11]:
# train model
model_history = model.fit(
    x=X_train,
    y=Y_train,
    epochs=1000,
    verbose=1
)

Epoch 1/1000
189/189 [==============================] - 1s 2ms/step - loss: 2.9759 - accuracy: 0.2485
Epoch 2/1000
189/189 [==============================] - 0s 2ms/step - loss: 2.1205 - accuracy: 0.5067
Epoch 3/1000
189/189 [==============================] - 0s 1ms/step - loss: 1.7340 - accuracy: 0.6270
Epoch 4/1000
189/189 [==============================] - 0s 1ms/step - loss: 1.6014 - accuracy: 0.6500
Epoch 5/1000
189/189 [==============================] - 0s 1ms/step - loss: 1.5304 - accuracy: 0.6661
Epoch 6/1000
189/189 [==============================] - 0s 1ms/step - loss: 1.4812 - accuracy: 0.6801
Epoch 7/1000
189/189 [==============================] - 0s 1ms/step - loss: 1.4411 - accuracy: 0.6931
Epoch 8/1000
189/189 [==============================] - 0s 1ms/step - loss: 1.4069 - accuracy: 0.7048
Epoch 9/1000
189/189 [==============================] - 0s 1ms/step - loss: 1.3743 - accuracy: 0.7109
Epoch 10/1000
189/189 [==============================] - 0s 2ms/step - loss: 1.346

In [12]:
# evaluate model
model.evaluate(X_train, Y_train, verbose=1)

189/189 [==============================] - 0s 1ms/step - loss: 0.6304 - accuracy: 0.8029


[0.6303772926330566, 0.8028822541236877]

In [13]:
# test model
x,_ = encode_dataset2(["носк"],ru_letters,['aaaaaaaaaaaaaaaaaaaaa'],en_letters)
predicted = model.predict(x)
output = ""
for c in predicted:
  output = output+  en_letters[list(c).index(max(c))]
output = output.replace(" ", "") 
output   

1/1 [==============================] - 0s 91ms/step


'nosk'

In [14]:
patterns[0:1000]

array(['and', 'for', 'that', 'this', 'with', 'you', 'not', 'are', 'from',
       'your', 'all', 'have', 'new', 'more', 'was', 'will', 'home', 'can',
       'about', 'page', 'has', 'search', 'free', 'but', 'our', 'one',
       'other', 'information', 'time', 'they', 'site', 'may', 'what',
       'which', 'their', 'news', 'out', 'use', 'any', 'there', 'see',
       'only', 'his', 'when', 'contact', 'here', 'business', 'who', 'web',
       'also', 'now', 'help', 'get', 'view', 'online', 'first', 'been',
       'would', 'how', 'were', 'services', 'some', 'these', 'click',
       'its', 'like', 'service', 'than', 'find', 'price', 'date', 'back',
       'top', 'people', 'had', 'list', 'name', 'just', 'over', 'state',
       'year', 'day', 'into', 'email', 'two', 'health', 'world', 'next',
       'used', 'work', 'last', 'most', 'products', 'music', 'buy', 'data',
       'make', 'them', 'should', 'product', 'system', 'post', 'her',
       'city', 'add', 'policy', 'number', 'such', 'please', 'a

In [15]:
# get closest matches
# галерея , фермер ,фигура,бар,план,порт,флаг,energy,карта,Паника,мотор     ----  белый - belly  ---- факультет ,,журнал
# patterns = ['car', 'fish', 'belly', 'boy','girl','gallary','give','land','wind','water','','faculty','wind','winter','home','television','suit','store','journal','neighbor','football','glass','grass','growth']

print('matched words:',get_close_matches(output,patterns))

matched words: ['nose', 'notes', 'nokia']


In [19]:
model.save("model.keras")
loaded_model = tf.keras.models.load_model("model.keras")